In [4]:
%%writefile scheme.py

"""
Numerical scheme (Section 5.1): CN diffusion + explicit reaction/taxis, Neumann (ghost reflection).

Model variables: S, R, I (diffuse); P, A (baseline: d_P=d_A=0, no diffusion).
- Neumann BC is enforced for diffusion via a ghost-reflection Neumann Laplacian.
- For taxis terms, Neumann-style ghost reflection is enforced via reflect-padding
  when computing gradients/divergences.

This file is intended as a reproducible reference implementation of the scheme
described in \Cref{subsec:numerical_scheme}.
"""

from __future__ import annotations

import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla


# -----------------------------
# 1) Baseline parameter set (Table 1)
# -----------------------------
P_BASE = {
    # Diffusion (baseline hybrid regime)
    "d_S": 1e-3,
    "d_R": 1e-3,
    "d_I": 5.0,        # fast-diffusing exogenous drug (single-dose)
    "d_P": 0.0,        # baseline: no diffusion
    "d_A": 0.0,        # baseline: no diffusion (set >0 in taxis experiments if needed)

    # Kinetics
    "lambda_S": 0.5,
    "lambda_R": 0.5,
    "K": 2.0,
    "alpha": 0.1,
    "xi": 0.1,

    # Drug-dependent terms
    "gamma_I": 1.0,
    "delta_0": 0.5,
    "K_I": 0.5,

    # Feedback strength (used in some extended regimes)
    "eta": 0.2,

    # Stromal conversion
    "theta": 1.0,
    "beta": 0.5,

    # Taxis sensitivities (set to 0 for base; vary in experiments)
    "chi_S": 0.0,
    "chi_R": 0.0,
}


# -----------------------------
# 2) Neumann operators (ghost reflection)
# -----------------------------
def neumann_laplacian_1d(N: int, h: float) -> sp.csr_matrix:
    """
    1D second-order Laplacian with homogeneous Neumann BC via ghost reflection:
      u_{-1} = u_{1}, u_{N} = u_{N-2}.

    Matrix approximates (d^2/dx^2) on a uniform grid.
    """
    e = np.ones(N)
    L = sp.diags([e, -2.0 * e, e], offsets=[-1, 0, 1], shape=(N, N), format="lil")

    # Ghost reflection modifies boundary neighbor coefficients:
    # i=0: (u1 - 2u0 + u_{-1})/h^2 = (2u1 - 2u0)/h^2  => L[0,1]=2
    # i=N-1: (u_N - 2u_{N-1} + u_{N-2})/h^2 = (2u_{N-2} - 2u_{N-1})/h^2 => L[N-1,N-2]=2
    if N >= 2:
        L[0, 1] = 2.0
        L[N - 1, N - 2] = 2.0

    return (L.tocsr()) / (h * h)


def neumann_laplacian_2d(Nx: int, Ny: int, h: float) -> sp.csr_matrix:
    """
    2D Neumann Laplacian on [0,1]^2 via Kronecker sum:
      L = I_y ⊗ L_x + L_y ⊗ I_x
    """
    Lx = neumann_laplacian_1d(Nx, h)
    Ly = neumann_laplacian_1d(Ny, h)
    Ix = sp.eye(Nx, format="csr")
    Iy = sp.eye(Ny, format="csr")
    return sp.kron(Iy, Lx, format="csr") + sp.kron(Ly, Ix, format="csr")


def grad_neumann(F: np.ndarray, h: float) -> tuple[np.ndarray, np.ndarray]:
    """
    Neumann-style gradient via reflect padding (ghost reflection).
    Returns (Fx, Fy) on the original grid.
    """
    Fp = np.pad(F, pad_width=1, mode="reflect")
    Fx = (Fp[1:-1, 2:] - Fp[1:-1, :-2]) / (2.0 * h)
    Fy = (Fp[2:, 1:-1] - Fp[:-2, 1:-1]) / (2.0 * h)
    return Fx, Fy


def div_neumann(Jx: np.ndarray, Jy: np.ndarray, h: float) -> np.ndarray:
    """
    Neumann-style divergence via reflect padding (ghost reflection).
    Returns div(J) on the original grid.
    """
    Jxp = np.pad(Jx, pad_width=1, mode="reflect")
    Jyp = np.pad(Jy, pad_width=1, mode="reflect")
    dJx = (Jxp[1:-1, 2:] - Jxp[1:-1, :-2]) / (2.0 * h)
    dJy = (Jyp[2:, 1:-1] - Jyp[:-2, 1:-1]) / (2.0 * h)
    return dJx + dJy


# -----------------------------
# 3) Nonlinearities and reactions
# -----------------------------
def phi(I: np.ndarray) -> np.ndarray:
    """Switch function phi(I) = tanh(5I)."""
    return np.tanh(5.0 * I)


def delta(I: np.ndarray, p: dict) -> np.ndarray:
    """Inhibition function delta(I) = delta_0 * I/(I + K_I)."""
    return p["delta_0"] * I / (I + p["K_I"] + 1e-12)


def compute_reactions(
    S: np.ndarray, R: np.ndarray, I: np.ndarray, P: np.ndarray, A: np.ndarray, p: dict
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Pointwise reaction terms G_S, G_R, G_I, G_P, G_A (explicitly treated).
    """
    phi_I = phi(I)
    delta_I = delta(I, p)
    sat = 1.0 - (S + R) / p["K"]

    G_S = (
        p["lambda_S"] * S * sat
        - p["alpha"] * S
        - delta_I * S
        + p["xi"] * (1.0 - phi_I) * R
    )

    G_R = (
        p["lambda_R"] * R * sat
        + p["alpha"] * S
        + p["eta"] * phi_I * A * R
        - p["xi"] * (1.0 - phi_I) * R
    )

    # Open-loop drug decay (single-dose, no source)
    G_I = -p["gamma_I"] * I

    # Baseline stromal conversion (ODE in baseline if d_P=d_A=0)
    G_P = -p["theta"] * phi_I * P + p["beta"] * (1.0 - phi_I) * A
    G_A =  p["theta"] * phi_I * P - p["beta"] * (1.0 - phi_I) * A

    return G_S, G_R, G_I, G_P, G_A


# -----------------------------
# 4) Taxis term (explicit)
# -----------------------------
def taxis_term(
    W: np.ndarray,
    signal: np.ndarray,
    chi: float,
    h: float,
    mode: str = "linear",
    alpha_sat: float = 0.0,
) -> np.ndarray:
    """
    Returns the explicit taxis contribution:  -div( J ), with J defined as:
      - mode="linear":      J = chi * W * grad(signal)
      - mode="flux_sat":    J = chi * W * grad(signal) / (1 + alpha_sat * |grad(signal)|)

    Neumann-style boundary handling uses reflect padding for grad/div.
    """
    if chi == 0.0:
        return np.zeros_like(W)

    sx, sy = grad_neumann(signal, h)

    if mode == "linear":
        Jx = chi * W * sx
        Jy = chi * W * sy

    elif mode == "flux_sat":
        if alpha_sat <= 0.0:
            raise ValueError("alpha_sat must be positive for flux_sat.")
        gnorm = np.sqrt(sx * sx + sy * sy)
        Jx = chi * W * (sx / (1.0 + alpha_sat * gnorm))
        Jy = chi * W * (sy / (1.0 + alpha_sat * gnorm))

    else:
        raise ValueError(f"Unknown taxis mode: {mode}")

    return -div_neumann(Jx, Jy, h)


# -----------------------------
# 5) Crank–Nicolson diffusion step (semi-implicit)
# -----------------------------
def cn_factorized(L: sp.csr_matrix, d: float, dt: float) -> tuple[callable, sp.csr_matrix]:
    """
    Build CN matrices for: (I - dt/2 d L) u^{n+1} = (I + dt/2 d L) u^n + dt * F^n
    Returns:
      solve_lhs: factorized solver for LHS
      rhs_mat: CSR matrix for RHS multiplication
    """
    N = L.shape[0]
    I = sp.eye(N, format="csr")
    A_lhs = (I - 0.5 * dt * d * L).tocsc()
    A_rhs = (I + 0.5 * dt * d * L).tocsr()
    solve_lhs = spla.factorized(A_lhs)
    return solve_lhs, A_rhs


# -----------------------------
# 6) Initialization near homogeneous equilibrium
# -----------------------------
def homogeneous_equilibrium(p: dict) -> tuple[float, float]:
    """
    A simple baseline equilibrium for (S,R) when I=0 and (P,A) held at (1,0):
      S* = (xi K)/(alpha + xi),  R* = (alpha K)/(alpha + xi).
    """
    S_star = (p["xi"] * p["K"]) / (p["alpha"] + p["xi"])
    R_star = (p["alpha"] * p["K"]) / (p["alpha"] + p["xi"])
    return S_star, R_star


def initialize_fields(
    Nx: int,
    Ny: int,
    p: dict,
    eps: float = 1e-3,
    seed: int = 42,
    P_T: float = 1.0,
) -> dict[str, np.ndarray]:
    """
    Initialize near homogeneous equilibrium with unbiased i.i.d. noise for S,R,I.
    Enforce homogeneous pool assumption P+A=P_T by taking P=P_T, A=0.
    """
    rng = np.random.default_rng(seed)
    S_star, R_star = homogeneous_equilibrium(p)

    S = S_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    R = R_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))

    # Single-dose drug: start nonnegative; allow small spatial heterogeneity if desired
    I = 0.0 + eps * rng.uniform(0.0, 1.0, size=(Ny, Nx))

    P = P_T * np.ones((Ny, Nx))
    A = np.zeros((Ny, Nx))

    # Enforce nonnegativity at t=0
    S = np.maximum(S, 0.0)
    R = np.maximum(R, 0.0)
    I = np.maximum(I, 0.0)

    return {"S": S, "R": R, "I": I, "P": P, "A": A}


# -----------------------------
# 7) One-step update (CN diffusion + explicit source)
# -----------------------------
def step_system(
    state: dict[str, np.ndarray],
    p: dict,
    h: float,
    dt: float,
    Nx: int,
    Ny: int,
    L2: sp.csr_matrix,
    solvers: dict[str, tuple[callable, sp.csr_matrix]],
    taxis: dict | None = None,
) -> dict[str, np.ndarray]:
    """
    Advances one time-step for the (S,R,I,P,A) system:
      - S,R,I: CN diffusion + explicit reaction/taxis
      - P,A: explicit Euler in baseline (d_P=d_A=0)

    taxis (optional) structure:
      taxis = {
        "S": {"signal": <array>, "chi": float, "mode": "linear"/"flux_sat", "alpha_sat": float},
        "R": {...},
      }
    """
    S, R, I, P, A = state["S"], state["R"], state["I"], state["P"], state["A"]

    # Explicit reactions
    G_S, G_R, G_I, G_P, G_A = compute_reactions(S, R, I, P, A, p)

    # Explicit taxis (if requested)
    C_S = np.zeros_like(S)
    C_R = np.zeros_like(R)
    if taxis is not None:
        if "S" in taxis:
            cfg = taxis["S"]
            C_S = taxis_term(S, cfg["signal"], cfg["chi"], h, cfg.get("mode", "linear"), cfg.get("alpha_sat", 0.0))
        if "R" in taxis:
            cfg = taxis["R"]
            C_R = taxis_term(R, cfg["signal"], cfg["chi"], h, cfg.get("mode", "linear"), cfg.get("alpha_sat", 0.0))

    # CN updates: flatten in row-major order
    def cn_update(var: np.ndarray, solve_lhs: callable, rhs_mat: sp.csr_matrix, F: np.ndarray) -> np.ndarray:
        rhs = rhs_mat.dot(var.reshape(-1)) + dt * F.reshape(-1)
        out = solve_lhs(rhs).reshape((Ny, Nx))
        return out

    solve_S, rhs_S = solvers["S"]
    solve_R, rhs_R = solvers["R"]
    solve_I, rhs_I = solvers["I"]

    S_new = cn_update(S, solve_S, rhs_S, G_S + C_S)
    R_new = cn_update(R, solve_R, rhs_R, G_R + C_R)
    I_new = cn_update(I, solve_I, rhs_I, G_I)

    # Explicit Euler for P,A in baseline (d_P=d_A=0)
    P_new = P + dt * G_P
    A_new = A + dt * G_A

    # Nonnegativity safeguard (optional, but consistent with the PDE invariance)
    S_new = np.maximum(S_new, 0.0)
    R_new = np.maximum(R_new, 0.0)
    I_new = np.maximum(I_new, 0.0)
    P_new = np.maximum(P_new, 0.0)
    A_new = np.maximum(A_new, 0.0)

    return {"S": S_new, "R": R_new, "I": I_new, "P": P_new, "A": A_new}


# -----------------------------
# 8) Example: build scheme objects (no plotting; experiments define figures)
# -----------------------------
if __name__ == "__main__":
    # Domain/grid
    L = 1.0
    Nx = Ny = 51
    h = L / (Nx - 1)

    # Time parameters (set by experiments)
    dt = 1e-2

    # Operators
    L2 = neumann_laplacian_2d(Nx, Ny, h)

    # CN solvers for diffusing variables
    solvers = {
        "S": cn_factorized(L2, P_BASE["d_S"], dt),
        "R": cn_factorized(L2, P_BASE["d_R"], dt),
        "I": cn_factorized(L2, P_BASE["d_I"], dt),
    }

    # Initialize
    state = initialize_fields(Nx, Ny, P_BASE, eps=1e-3, seed=42, P_T=1.0)

    # One step (base model: no taxis)
    state = step_system(
        state=state,
        p=P_BASE,
        h=h,
        dt=dt,
        Nx=Nx,
        Ny=Ny,
        L2=L2,
        solvers=solvers,
        taxis=None,
    )

    print("One step completed.")
    print(f"Max S={state['S'].max():.6f}, Max R={state['R'].max():.6f}, Max I={state['I'].max():.6f}")

Overwriting scheme.py


In [6]:
"""
experiments.py
Reproducible numerical experiments for Section 5 (Regimes I, I′, II–III).

Assumes you saved the scheme/engine code as: scheme.py
(from my previous message: Neumann operators + CN step + explicit taxis).

This script:
  (1) Base model: relaxation to homogeneity (Regime I)
  (2) Unidirectional chemotaxis toward an open-loop, relaxing signal A (Regime I′)
  (3) Bidirectional feedback via chemoattractant c:
        - Regime III: ill-posed/aggregation (unregularized / linear taxis)
        - Regime II: finite-band patterns (flux-saturated taxis / velocity limiting)

All runs:
  - Neumann BC via ghost reflection for diffusion and reflect-padding for taxis fluxes.
  - Fixed RNG seeds for reproducibility.
  - Saves figures under ./figures/

You can tune parameters in the PARAM blocks below to match your paper/figures.
"""

from __future__ import annotations

import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp

from scheme import (
    P_BASE,
    neumann_laplacian_2d,
    cn_factorized,
    initialize_fields,
    step_system,
    homogeneous_equilibrium,
    grad_neumann,
    div_neumann,
    taxis_term,
)

# -----------------------------
# 0) I/O helpers
# -----------------------------
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)


def savefig(path: str) -> None:
    plt.tight_layout()
    plt.savefig(path, dpi=250)
    plt.close()


def l2_deviation(field: np.ndarray, const_value: float, h: float) -> float:
    diff = field - const_value
    return float(np.sqrt(np.sum(diff * diff) * h * h))


def summed_variance(fields: dict[str, np.ndarray]) -> float:
    # consistent with Sigma_var definition, using discrete variance
    return float(sum(np.var(v) for v in fields.values()))


# -----------------------------
# 1) Shared discretization
# -----------------------------
L = 1.0
Nx = Ny = 51
h = L / (Nx - 1)

# time step chosen to match your paper prose; adjust per stability needs
dt = 1e-2


def build_solvers(L2: sp.csr_matrix, p: dict, dt: float, include: tuple[str, ...]) -> dict[str, tuple[callable, sp.csr_matrix]]:
    solvers = {}
    if "S" in include:
        solvers["S"] = cn_factorized(L2, p["d_S"], dt)
    if "R" in include:
        solvers["R"] = cn_factorized(L2, p["d_R"], dt)
    if "I" in include:
        solvers["I"] = cn_factorized(L2, p["d_I"], dt)
    if "A" in include:
        solvers["A"] = cn_factorized(L2, p["d_A"], dt)
    if "c" in include:
        solvers["c"] = cn_factorized(L2, p["D_c"], dt)
    return solvers


# -----------------------------
# 2) Unidirectional signal A: open-loop relaxer
# -----------------------------
def step_open_loop_signal_A(
    A: np.ndarray,
    solvers_A: tuple[callable, sp.csr_matrix],
    d_A: float,
    gamma_A: float,
    dt: float,
    Ny: int,
    Nx: int,
) -> np.ndarray:
    """
    Open-loop signal dynamics:  ∂_t A = d_A ΔA - gamma_A A
    advanced with CN diffusion + explicit decay.
    """
    solve_A, rhs_A = solvers_A
    G_A = -gamma_A * A
    rhs = rhs_A.dot(A.reshape(-1)) + dt * G_A.reshape(-1)
    A_new = solve_A(rhs).reshape((Ny, Nx))
    return np.maximum(A_new, 0.0)


# -----------------------------
# 3) Bidirectional chemoattractant c (feedback loop)
# -----------------------------
def step_chemoattractant_c(
    c: np.ndarray,
    S: np.ndarray,
    solvers_c: tuple[callable, sp.csr_matrix],
    kappa: float,
    rho: float,
    dt: float,
    Ny: int,
    Nx: int,
) -> np.ndarray:
    """
    Feedback signal:  ∂_t c = D_c Δc + kappa*S - rho*c
    advanced with CN diffusion + explicit reaction.
    """
    solve_c, rhs_c = solvers_c
    G_c = kappa * S - rho * c
    rhs = rhs_c.dot(c.reshape(-1)) + dt * G_c.reshape(-1)
    c_new = solve_c(rhs).reshape((Ny, Nx))
    return np.maximum(c_new, 0.0)


# -----------------------------
# 4) Experiment (Regime I): Base relaxation
# -----------------------------
def run_regime_I_base(T: float = 5.0, eps: float = 1e-3, seed: int = 42) -> None:
    p = dict(P_BASE)
    p["chi_S"] = 0.0
    p["chi_R"] = 0.0

    L2 = neumann_laplacian_2d(Nx, Ny, h)
    solvers = build_solvers(L2, p, dt, include=("S", "R", "I"))

    state = initialize_fields(Nx, Ny, p, eps=eps, seed=seed, P_T=1.0)
    S_star, R_star = homogeneous_equilibrium(p)

    nsteps = int(T / dt)
    times = []
    ES = []
    ER = []
    EI = []

    snapshots = {}
    snap_times = [0.0, T / 2.0, T]
    snap_idx = {int(t / dt): t for t in snap_times}

    for n in range(nsteps + 1):
        t = n * dt
        times.append(t)
        ES.append(l2_deviation(state["S"], S_star, h))
        ER.append(l2_deviation(state["R"], R_star, h))
        EI.append(l2_deviation(state["I"], 0.0, h))

        if n in snap_idx:
            snapshots[snap_idx[n]] = state["S"].copy()

        if n < nsteps:
            state = step_system(
                state=state,
                p=p,
                h=h,
                dt=dt,
                Nx=Nx,
                Ny=Ny,
                L2=L2,
                solvers=solvers,
                taxis=None,
            )

    # Figure: S snapshots
    fig, axes = plt.subplots(1, 3, figsize=(12, 3.6))
    for ax, t_snap in zip(axes, snap_times):
        im = ax.imshow(snapshots[t_snap], origin="lower", extent=[0, 1, 0, 1])
        ax.set_title(f"S at t={t_snap:.2f}")
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.03)
    plt.suptitle("Regime I (Base): Progressive homogenization")
    savefig(os.path.join(FIGDIR, "regimeI_base_snapshots.png"))

    # Figure: L2 decay
    plt.figure(figsize=(6.2, 4.2))
    plt.semilogy(times, ES, label=r"$E_S(t)$")
    plt.semilogy(times, ER, label=r"$E_R(t)$", linestyle="--")
    plt.semilogy(times, EI, label=r"$E_I(t)$", linestyle="-.")
    plt.xlabel("t")
    plt.ylabel("L2 deviation")
    plt.title("Regime I (Base): Exponential relaxation (semi-log)")
    plt.legend()
    plt.grid(True, which="both", alpha=0.3)
    savefig(os.path.join(FIGDIR, "regimeI_base_L2.png"))


# -----------------------------
# 5) Experiment (Regime I′): Unidirectional chemotaxis toward relaxing open-loop A
# -----------------------------
def run_regime_Iprime_oneway(
    T: float = 5.0,
    eps: float = 1e-3,
    seed: int = 42,
    chi: float = 0.5,
    d_A: float = 1e-4,
    gamma_A: float = 1.0,
    taxis_mode: str = "linear",
    alpha_sat: float = 0.0,
) -> None:
    """
    Oneway: S,R drift up ∇A, but A is open-loop and relaxes (diffuse+decay),
    so no sustained pattern generation from homogeneous equilibrium.
    """
    p = dict(P_BASE)
    p["chi_S"] = chi
    p["chi_R"] = chi
    p["d_A"] = d_A  # to make ∇A well-defined numerically

    L2 = neumann_laplacian_2d(Nx, Ny, h)
    solvers = build_solvers(L2, p, dt, include=("S", "R", "I", "A"))

    # initialize S,R,I,P,A
    state = initialize_fields(Nx, Ny, p, eps=eps, seed=seed, P_T=1.0)

    # overwrite A with a smooth bump + small noise (open-loop signal landscape)
    rng = np.random.default_rng(seed + 7)
    X = np.linspace(0, 1, Nx)
    Y = np.linspace(0, 1, Ny)
    XX, YY = np.meshgrid(X, Y)
    bump = np.exp(-((XX - 0.5) ** 2 + (YY - 0.5) ** 2) / (2 * 0.12**2))
    state["A"] = np.maximum(0.2 * bump + 0.02 * rng.uniform(-1, 1, size=(Ny, Nx)), 0.0)

    S_star, R_star = homogeneous_equilibrium(p)

    nsteps = int(T / dt)
    times = []
    ES = []

    for n in range(nsteps + 1):
        t = n * dt
        times.append(t)
        ES.append(l2_deviation(state["S"], S_star, h))

        if n < nsteps:
            # update open-loop signal A first
            state["A"] = step_open_loop_signal_A(
                A=state["A"],
                solvers_A=solvers["A"],
                d_A=p["d_A"],
                gamma_A=gamma_A,
                dt=dt,
                Ny=Ny,
                Nx=Nx,
            )

            taxis = {
                "S": {"signal": state["A"], "chi": p["chi_S"], "mode": taxis_mode, "alpha_sat": alpha_sat},
                "R": {"signal": state["A"], "chi": p["chi_R"], "mode": taxis_mode, "alpha_sat": alpha_sat},
            }

            # Use the same step_system, but (temporarily) store A inside state
            state = step_system(
                state=state,
                p=p,
                h=h,
                dt=dt,
                Nx=Nx,
                Ny=Ny,
                L2=L2,
                solvers={"S": solvers["S"], "R": solvers["R"], "I": solvers["I"]},
                taxis=taxis,
            )

    # Figure: L2 deviation (should show transient focusing then decay)
    plt.figure(figsize=(6.2, 4.2))
    plt.semilogy(times, ES, label=r"$E_S(t)$")
    plt.xlabel("t")
    plt.ylabel("L2 deviation of S")
    plt.title("Regime I′ (Oneway): transient focusing, then relaxation")
    plt.legend()
    plt.grid(True, which="both", alpha=0.3)
    savefig(os.path.join(FIGDIR, "regimeIprime_oneway_L2.png"))


# -----------------------------
# 6) Experiments (Regimes II–III): Bidirectional feedback via c
# -----------------------------
def run_bidirectional_feedback(
    T: float,
    seed: int,
    eps: float,
    # c dynamics
    D_c: float,
    kappa: float,
    rho: float,
    # taxis of S toward c
    chi_c: float,
    taxis_mode: str,
    alpha_sat: float,
    # stability cutoffs
    blowup_threshold: float = 1e6,
    snapshot_times: tuple[float, ...] = (0.0, 0.5, 1.0),
) -> tuple[dict[str, np.ndarray], dict[float, np.ndarray], dict[str, list[float]]]:
    """
    Runs S,R,I with feedback c. c is produced by S (kappa) and decays (rho).
    S undergoes taxis toward c using either:
      - taxis_mode="linear"   (Regime III: prone to collapse)
      - taxis_mode="flux_sat" (Regime II: velocity-limited; finite-band patterns)

    Returns: (final_state, snapshots_of_S, history)
    """
    p = dict(P_BASE)
    p["chi_S"] = 0.0
    p["chi_R"] = 0.0
    p["D_c"] = D_c

    L2 = neumann_laplacian_2d(Nx, Ny, h)

    # CN solvers for S,R,I and c
    solvers = build_solvers(L2, p, dt, include=("S", "R", "I", "c"))

    # initialize base fields
    state = initialize_fields(Nx, Ny, p, eps=eps, seed=seed, P_T=1.0)

    # initialize c around its homogeneous steady state, perturbed
    S_star, _ = homogeneous_equilibrium(p)
    c_star = (kappa * S_star) / max(rho, 1e-12)
    rng = np.random.default_rng(seed + 11)
    c = np.maximum(c_star + eps * rng.uniform(-1, 1, size=(Ny, Nx)), 0.0)

    nsteps = int(T / dt)
    snapshots = {}
    snap_idx = {int(t / dt): t for t in snapshot_times}

    hist = {"t": [], "maxS": [], "maxc": []}

    for n in range(nsteps + 1):
        t = n * dt
        hist["t"].append(t)
        hist["maxS"].append(float(np.max(state["S"])))
        hist["maxc"].append(float(np.max(c)))

        if n in snap_idx:
            snapshots[snap_idx[n]] = state["S"].copy()

        # stop if blow-up
        if not np.isfinite(state["S"]).all() or np.max(state["S"]) > blowup_threshold:
            break

        if n < nsteps:
            # update c (CN diffusion + explicit reaction)
            c = step_chemoattractant_c(
                c=c,
                S=state["S"],
                solvers_c=solvers["c"],
                kappa=kappa,
                rho=rho,
                dt=dt,
                Ny=Ny,
                Nx=Nx,
            )

            taxis = {
                "S": {"signal": c, "chi": chi_c, "mode": taxis_mode, "alpha_sat": alpha_sat},
            }

            state = step_system(
                state=state,
                p=p,
                h=h,
                dt=dt,
                Nx=Nx,
                Ny=Ny,
                L2=L2,
                solvers={"S": solvers["S"], "R": solvers["R"], "I": solvers["I"]},
                taxis=taxis,
            )

    final = dict(state)
    final["c"] = c
    return final, snapshots, hist


def run_regime_III_collapse() -> None:
    """
    Regime III: unregularized (linear) taxis with strong feedback.
    Expect rapid amplification and numerical overflow/instability.
    """
    final, snaps, hist = run_bidirectional_feedback(
        T=2.0,
        seed=100,
        eps=5e-2,
        D_c=1e-2,
        kappa=2.0,
        rho=0.1,
        chi_c=5.0,              # strong drift
        taxis_mode="linear",    # no regularization
        alpha_sat=0.0,
        blowup_threshold=1e8,
        snapshot_times=(0.0, 0.5, 1.0),
    )

    # Plot max(S) over time
    plt.figure(figsize=(6.2, 4.2))
    plt.semilogy(hist["t"], np.maximum(hist["maxS"], 1e-16))
    plt.xlabel("t")
    plt.ylabel("max S(t)")
    plt.title("Regime III (Bidirectional, unregularized): blow-up indicator")
    plt.grid(True, which="both", alpha=0.3)
    savefig(os.path.join(FIGDIR, "regimeIII_blowup_maxS.png"))

    # Snapshot montage if available
    if len(snaps) >= 2:
        fig, axes = plt.subplots(1, len(snaps), figsize=(4.2 * len(snaps), 3.6))
        if len(snaps) == 1:
            axes = [axes]
        for ax, (ts, Ssnap) in zip(axes, sorted(snaps.items(), key=lambda kv: kv[0])):
            im = ax.imshow(Ssnap, origin="lower", extent=[0, 1, 0, 1])
            ax.set_title(f"S at t={ts:.2f}")
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.03)
        plt.suptitle("Regime III: aggregation/collapse snapshots")
        savefig(os.path.join(FIGDIR, "regimeIII_snapshots.png"))


def run_regime_II_patterns_flux_saturation() -> None:
    """
    Regime II: flux-saturated taxis (velocity limiting).
    This implements the *flux saturation* choice you requested:
      J = chi * S * grad(c) / (1 + alpha_c * |grad(c)|)

    alpha_sat = alpha_c (dimensionless in this discretization; tune in experiments).
    """
    final, snaps, hist = run_bidirectional_feedback(
        T=20.0,
        seed=101,
        eps=5e-2,
        D_c=1e-2,
        kappa=0.2,
        rho=0.2,
        chi_c=1.0,
        taxis_mode="flux_sat",
        alpha_sat=5.0,          # default: moderate velocity limiting
        blowup_threshold=1e6,
        snapshot_times=(0.0, 10.0, 20.0),
    )

    S = final["S"]
    c = final["c"]

    # Figure: final S and c + cross-section
    plt.figure(figsize=(12.8, 4.1))

    plt.subplot(1, 3, 1)
    im1 = plt.imshow(S, origin="lower", extent=[0, 1, 0, 1])
    plt.colorbar(im1, fraction=0.046, pad=0.03)
    plt.title("S (Regime II: patterns)")

    plt.subplot(1, 3, 2)
    im2 = plt.imshow(c, origin="lower", extent=[0, 1, 0, 1])
    plt.colorbar(im2, fraction=0.046, pad=0.03)
    plt.title("c (feedback signal)")

    plt.subplot(1, 3, 3)
    mid = Ny // 2
    plt.plot(np.linspace(0, 1, Nx), S[mid, :], label="S slice")
    # rescale c slice for overlay
    cs = c[mid, :]
    cs_scaled = (cs - cs.min()) / (cs.max() - cs.min() + 1e-12)
    cs_scaled = cs_scaled * (S.max() - S.min()) + S.min()
    plt.plot(np.linspace(0, 1, Nx), cs_scaled, "--", label="c slice (scaled)")
    plt.title("Cross-section (y=0.5)")
    plt.legend()
    plt.grid(True, alpha=0.3)

    savefig(os.path.join(FIGDIR, "regimeII_patterns_flux_sat.png"))

    # Figure: max S plateau (boundedness check)
    plt.figure(figsize=(6.2, 4.2))
    plt.plot(hist["t"], hist["maxS"])
    plt.xlabel("t")
    plt.ylabel("max S(t)")
    plt.title("Regime II (flux saturation): boundedness / plateau check")
    plt.grid(True, alpha=0.3)
    savefig(os.path.join(FIGDIR, "regimeII_maxS_plateau.png"))


# -----------------------------
# 9) Main entry: run all experiments used in Section 5
# -----------------------------
if __name__ == "__main__":
    # Regime I
    run_regime_I_base(T=5.0, eps=1e-3, seed=42)

    # Regime I′ (unidirectional, open-loop signal relaxes)
    run_regime_Iprime_oneway(
        T=5.0,
        eps=1e-3,
        seed=42,
        chi=0.5,
        d_A=1e-4,
        gamma_A=1.0,
        taxis_mode="linear",
    )

    # Regime III (collapse / ill-posed indicator)
    run_regime_III_collapse()

    # Regime II (finite-band patterns with flux saturation / velocity limiting)
    run_regime_II_patterns_flux_saturation()

    print(f"Done. Figures saved to: {FIGDIR}/")

Done. Figures saved to: figures/


In [7]:
"""
code_regimeI_base.py

Reproducible code for:
  \subsection{Base model: relaxation to homogeneity (Regime I)}
  \label{subsec:numerical_base_regimeI}

Generates (and saves) the two figure assets that your subsection references:
  - Figure \label{fig:homogeneous}: S snapshots at t = 0, 2.5, 5.0
  - Figure \label{fig:norm_evolution}: L^2 deviation curves (base model = blue solid)

Design choices (submission-grade consistency):
  - Symbols match paper: (S,R,I,P,A), gamma_I, d_I, K_I, Neumann BC.
  - Neumann BC for diffusion via ghost-point reflection (Kronecker Laplacian).
  - Base regime uses d_P = d_A = 0; P,A advanced by explicit ODE at each gridpoint.
  - CN diffusion + explicit reactions (as stated in your Section 5.1).
  - Fixed random seed for reproducibility.
  - Saves figures to ./figures with deterministic filenames.

Run:
  python code_regimeI_base.py

Outputs:
  figures/fig_homogeneous.png   (use as Unknown-120 replacement)
  figures/fig_norm_evolution.png (use as Unknown-121 replacement)
"""

from __future__ import annotations
import os
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt


# ----------------------------
# 0) I/O
# ----------------------------
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

FIG_HOMOGENEOUS = os.path.join(FIGDIR, "fig_homogeneous.png")
FIG_NORM_EVOLUTION = os.path.join(FIGDIR, "fig_norm_evolution.png")


def savefig(path: str) -> None:
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.close()


# ----------------------------
# 1) Parameters (match Table 1 baseline)
# ----------------------------
p = {
    # Diffusion (base: only S,R,I diffuse)
    "d_S": 1e-3,
    "d_R": 1e-3,
    "d_I": 5.0,

    # Baseline: no diffusion for P,A in Regime I
    "d_P": 0.0,
    "d_A": 0.0,

    # Kinetics
    "lambda_S": 0.5,
    "lambda_R": 0.5,
    "K": 2.0,
    "alpha": 0.1,
    "xi": 0.1,

    # Drug (single-dose open-loop)
    "gamma_I": 1.0,
    "delta_0": 0.5,
    "K_I": 0.5,

    # Microenvironment conversion and feedback
    "eta": 0.2,
    "theta": 1.0,
    "beta": 0.5,
}


def phi(I: np.ndarray) -> np.ndarray:
    return np.tanh(5.0 * I)


def delta(I: np.ndarray, p: dict) -> np.ndarray:
    return p["delta_0"] * I / (I + p["K_I"] + 1e-12)


# ----------------------------
# 2) Domain / grid
# ----------------------------
L = 1.0
Nx = Ny = 51  # matches your revised draft default
h = L / (Nx - 1)

# time
dt = 1e-2
T_final = 5.0
snapshot_times = (0.0, 2.5, 5.0)

# reproducibility
seed = 42
eps = 1e-3


# ----------------------------
# 3) Neumann Laplacian via ghost reflection (Kronecker)
# ----------------------------
def neumann_laplacian_1d(N: int, h: float) -> sp.csr_matrix:
    """
    1D second-order Laplacian with homogeneous Neumann BCs using ghost reflection:
      u_{-1}=u_1, u_{N}=u_{N-2}
    """
    e = np.ones(N)
    A = sp.diags([e, -2 * e, e], [-1, 0, 1], shape=(N, N), format="lil")
    # boundary reflection modifies the adjacent entries
    A[0, 1] = 2.0
    A[N - 1, N - 2] = 2.0
    return (A.tocsr()) / (h * h)


def neumann_laplacian_2d(Nx: int, Ny: int, h: float) -> sp.csr_matrix:
    Ax = neumann_laplacian_1d(Nx, h)
    Ay = neumann_laplacian_1d(Ny, h)
    Ix = sp.eye(Nx, format="csr")
    Iy = sp.eye(Ny, format="csr")
    # row-major flattening: kron(Iy, Ax) + kron(Ay, Ix)
    return sp.kron(Iy, Ax, format="csr") + sp.kron(Ay, Ix, format="csr")


L_h = neumann_laplacian_2d(Nx, Ny, h)
N = Nx * Ny
Isp = sp.eye(N, format="csr")


def cn_factorized(Lh: sp.csr_matrix, d: float, dt: float) -> tuple[callable, sp.csr_matrix]:
    """
    Returns (solve, RHS_mat) for Crank–Nicolson:
      (I - dt/2 d L) u^{n+1} = (I + dt/2 d L) u^n + dt F^n
    """
    lhs = (Isp - 0.5 * dt * d * Lh).tocsc()
    rhs = (Isp + 0.5 * dt * d * Lh).tocsr()
    solve = spla.factorized(lhs)
    return solve, rhs


solve_S, rhs_S = cn_factorized(L_h, p["d_S"], dt)
solve_R, rhs_R = cn_factorized(L_h, p["d_R"], dt)
solve_I, rhs_I = cn_factorized(L_h, p["d_I"], dt)


# ----------------------------
# 4) Homogeneous equilibrium (used in E_W(t))
# ----------------------------
def homogeneous_equilibrium(p: dict) -> tuple[float, float]:
    """
    Uses the paper’s base equilibrium at I*=0 with switching balance:
      S* = xi K/(alpha+xi), R* = alpha K/(alpha+xi)
    """
    S_star = (p["xi"] * p["K"]) / (p["alpha"] + p["xi"])
    R_star = (p["alpha"] * p["K"]) / (p["alpha"] + p["xi"])
    return float(S_star), float(R_star)


S_star, R_star = homogeneous_equilibrium(p)
I_star = 0.0
P_star = 1.0
A_star = 0.0


# ----------------------------
# 5) Reactions (pointwise)
# ----------------------------
def compute_reactions(S: np.ndarray, R: np.ndarray, I: np.ndarray, P: np.ndarray, A: np.ndarray, p: dict):
    phi_I = phi(I)
    delta_I = delta(I, p)
    sat = 1.0 - (S + R) / p["K"]

    G_S = (
        p["lambda_S"] * S * sat
        - p["alpha"] * S
        - delta_I * S
        + p["xi"] * (1.0 - phi_I) * R
    )

    G_R = (
        p["lambda_R"] * R * sat
        + p["alpha"] * S
        + p["eta"] * phi_I * A * R
        - p["xi"] * (1.0 - phi_I) * R
    )

    G_I = -p["gamma_I"] * I

    # base regime: ODEs per grid point (d_P=d_A=0)
    G_P = -p["theta"] * phi_I * P + p["beta"] * (1.0 - phi_I) * A
    G_A =  p["theta"] * phi_I * P - p["beta"] * (1.0 - phi_I) * A

    return G_S, G_R, G_I, G_P, G_A


# ----------------------------
# 6) Norms / metrics
# ----------------------------
def l2_deviation(field: np.ndarray, const_value: float, h: float) -> float:
    diff = field - const_value
    return float(np.sqrt(np.sum(diff * diff) * h * h))


# ----------------------------
# 7) Initialize (unbiased perturbations about equilibrium)
# ----------------------------
rng = np.random.default_rng(seed)

S = S_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
R = R_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
I = I_star + eps * rng.uniform(0.0, 1.0, size=(Ny, Nx))  # enforce I>=0
P = P_star * np.ones((Ny, Nx))
A = A_star * np.ones((Ny, Nx))


# ----------------------------
# 8) Time loop
# ----------------------------
nsteps = int(round(T_final / dt))
snap_indices = {int(round(t / dt)): t for t in snapshot_times}
snapshots_S: dict[float, np.ndarray] = {}

times = []
E_S = []
E_R = []
E_I = []

for n in range(nsteps + 1):
    t = n * dt
    times.append(t)
    E_S.append(l2_deviation(S, S_star, h))
    E_R.append(l2_deviation(R, R_star, h))
    E_I.append(l2_deviation(I, I_star, h))

    if n in snap_indices:
        snapshots_S[snap_indices[n]] = S.copy()

    if n == nsteps:
        break

    # explicit reactions
    G_S, G_R, G_I, G_P, G_A = compute_reactions(S, R, I, P, A, p)

    # CN diffusion for S,R,I
    S_flat = S.reshape(-1)
    R_flat = R.reshape(-1)
    I_flat = I.reshape(-1)

    rhs_vec_S = rhs_S.dot(S_flat) + dt * G_S.reshape(-1)
    rhs_vec_R = rhs_R.dot(R_flat) + dt * G_R.reshape(-1)
    rhs_vec_I = rhs_I.dot(I_flat) + dt * G_I.reshape(-1)

    S = solve_S(rhs_vec_S).reshape((Ny, Nx))
    R = solve_R(rhs_vec_R).reshape((Ny, Nx))
    I = solve_I(rhs_vec_I).reshape((Ny, Nx))

    # ODE updates for P,A (base regime)
    P = P + dt * G_P
    A = A + dt * G_A

    # positivity safeguard (optional but harmless for reproducibility)
    S = np.maximum(S, 0.0)
    R = np.maximum(R, 0.0)
    I = np.maximum(I, 0.0)
    P = np.maximum(P, 0.0)
    A = np.maximum(A, 0.0)


# ----------------------------
# 9) Figure \label{fig:homogeneous}: snapshots of S
# ----------------------------
fig, axes = plt.subplots(1, 3, figsize=(12, 3.8))
for ax, t_snap in zip(axes, snapshot_times):
    im = ax.imshow(
        snapshots_S[t_snap],
        origin="lower",
        extent=[0, 1, 0, 1],
    )
    ax.set_title(f"$S$ at $t={t_snap:.1f}$")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.03)
plt.suptitle("Base model: progressive homogenization (Neumann BC)")
savefig(FIG_HOMOGENEOUS)

# ----------------------------
# 10) Figure \label{fig:norm_evolution}: L2 deviation curves
# ----------------------------
plt.figure(figsize=(6.4, 4.4))
plt.semilogy(times, E_S, label="Base: $E_S(t)$", linewidth=2)  # blue solid by default
plt.semilogy(times, E_R, label="Base: $E_R(t)$", linewidth=2, linestyle="--")
plt.semilogy(times, E_I, label="Base: $E_I(t)$", linewidth=2, linestyle="-.")
plt.xlabel("t")
plt.ylabel(r"$\|W(\cdot,t)-W^*\|_{L^2(U)}$")
plt.title("Base model: exponential relaxation (semi-log)")
plt.grid(True, which="both", alpha=0.3)
plt.legend()
savefig(FIG_NORM_EVOLUTION)

print(f"Saved: {FIG_HOMOGENEOUS}")
print(f"Saved: {FIG_NORM_EVOLUTION}")

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1865197812.py:5: SyntaxWarning: invalid escape sequence '\s'
  \subsection{Base model: relaxation to homogeneity (Regime I)}


Saved: figures/fig_homogeneous.png
Saved: figures/fig_norm_evolution.png


In [8]:
"""
code_regimeIprime_oneway.py

Reproducible code for:
  \subsection{Unidirectional chemotaxis: transient focusing without generation (Regime I')}
  \label{subsec:numerical_base_oneway}

Goal:
  Produce the *red dashed* curve in Figure \label{fig:norm_evolution} by running the
  unidirectional chemotaxis extension (open-loop signal), showing:
    - possible transient rise in E_S(t)
    - eventual exponential decay (no sustained patterns)

Submission-grade consistency:
  - Symbols: (S,R,I,P,A), gamma_I, d_I, K_I.
  - Neumann BC: diffusion via ghost-point reflection Kronecker Laplacian;
    taxis flux uses reflect-padding gradient/divergence so the "ghost reflection"
    claim applies to chemotaxis as well.
  - Open-loop signal: A evolves independently of S,R taxis (no feedback loop).
    To create gradients to climb and to permit relaxation of the signal, we
    evolve A by a *simple* diffusive decay:
        A_t = d_A ΔA - gamma_A A
    This is consistent with your paper’s “open-loop signal relaxes” language.
    If your Eq. (oneway_damping) specifies a different open-loop signal dynamics,
    swap the A-update accordingly (the rest of the code stays the same).

Outputs:
  figures/fig_norm_evolution_oneway_overlay.png
    - overlays base (blue solid) vs one-way (red dashed) for E_S(t)

If you already generated base-model curves with code_regimeI_base.py,
this script will (by default) re-run the base model so the overlay is
self-contained and reproducible.

Run:
  python code_regimeIprime_oneway.py
"""

from __future__ import annotations
import os
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt


# ----------------------------
# 0) I/O
# ----------------------------
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

FIG_NORM_OVERLAY = os.path.join(FIGDIR, "fig_norm_evolution_oneway_overlay.png")


def savefig(path: str) -> None:
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.close()


# ----------------------------
# 1) Parameters (match Table 1 baseline; add one-way taxis + signal relaxation)
# ----------------------------
p = {
    # Diffusion (S,R,I diffuse)
    "d_S": 1e-3,
    "d_R": 1e-3,
    "d_I": 5.0,

    # Baseline: P,A no diffusion; in one-way chemotaxis we give A small diffusion
    # to make gradients well-defined (consistent with your Section 5 wording).
    "d_P": 0.0,
    "d_A": 1e-4,   # small regularization diffusion for signal A (open-loop)

    # Kinetics
    "lambda_S": 0.5,
    "lambda_R": 0.5,
    "K": 2.0,
    "alpha": 0.1,
    "xi": 0.1,

    # Drug (single-dose open-loop)
    "gamma_I": 1.0,
    "delta_0": 0.5,
    "K_I": 0.5,

    # Microenvironment conversion and feedback
    "eta": 0.2,
    "theta": 1.0,
    "beta": 0.5,

    # Unidirectional chemotaxis sensitivities (as in your draft example)
    "chi_S": 0.5,
    "chi_R": 0.5,

    # Open-loop signal relaxation (A_t = d_A ΔA - gamma_A A)
    "gamma_A": 1.0,
}


def phi(I: np.ndarray) -> np.ndarray:
    return np.tanh(5.0 * I)


def delta(I: np.ndarray, p: dict) -> np.ndarray:
    return p["delta_0"] * I / (I + p["K_I"] + 1e-12)


# ----------------------------
# 2) Domain / grid
# ----------------------------
L = 1.0
Nx = Ny = 51
h = L / (Nx - 1)

dt = 1e-2
T_final = 5.0

seed = 42
eps = 1e-3


# ----------------------------
# 3) Neumann Laplacian (ghost reflection)
# ----------------------------
def neumann_laplacian_1d(N: int, h: float) -> sp.csr_matrix:
    e = np.ones(N)
    A = sp.diags([e, -2 * e, e], [-1, 0, 1], shape=(N, N), format="lil")
    A[0, 1] = 2.0
    A[N - 1, N - 2] = 2.0
    return (A.tocsr()) / (h * h)


def neumann_laplacian_2d(Nx: int, Ny: int, h: float) -> sp.csr_matrix:
    Ax = neumann_laplacian_1d(Nx, h)
    Ay = neumann_laplacian_1d(Ny, h)
    Ix = sp.eye(Nx, format="csr")
    Iy = sp.eye(Ny, format="csr")
    return sp.kron(Iy, Ax, format="csr") + sp.kron(Ay, Ix, format="csr")


L_h = neumann_laplacian_2d(Nx, Ny, h)
N = Nx * Ny
Isp = sp.eye(N, format="csr")


def cn_factorized(Lh: sp.csr_matrix, d: float, dt: float) -> tuple[callable, sp.csr_matrix]:
    lhs = (Isp - 0.5 * dt * d * Lh).tocsc()
    rhs = (Isp + 0.5 * dt * d * Lh).tocsr()
    solve = spla.factorized(lhs)
    return solve, rhs


# ----------------------------
# 4) Neumann-consistent grad/div for taxis (reflect padding)
# ----------------------------
def grad_neumann(F: np.ndarray, h: float) -> tuple[np.ndarray, np.ndarray]:
    Fp = np.pad(F, pad_width=1, mode="reflect")
    Fx = (Fp[1:-1, 2:] - Fp[1:-1, :-2]) / (2 * h)
    Fy = (Fp[2:, 1:-1] - Fp[:-2, 1:-1]) / (2 * h)
    return Fx, Fy


def div_neumann(Jx: np.ndarray, Jy: np.ndarray, h: float) -> np.ndarray:
    Jxp = np.pad(Jx, pad_width=1, mode="reflect")
    Jyp = np.pad(Jy, pad_width=1, mode="reflect")
    dJx = (Jxp[1:-1, 2:] - Jxp[1:-1, :-2]) / (2 * h)
    dJy = (Jyp[2:, 1:-1] - Jyp[:-2, 1:-1]) / (2 * h)
    return dJx + dJy


def chemotaxis_linear(W: np.ndarray, Signal: np.ndarray, chi: float, h: float) -> np.ndarray:
    if chi == 0.0:
        return np.zeros_like(W)
    Sx, Sy = grad_neumann(Signal, h)
    Jx = chi * W * Sx
    Jy = chi * W * Sy
    return -div_neumann(Jx, Jy, h)


# ----------------------------
# 5) Base equilibrium
# ----------------------------
def homogeneous_equilibrium(p: dict) -> tuple[float, float]:
    S_star = (p["xi"] * p["K"]) / (p["alpha"] + p["xi"])
    R_star = (p["alpha"] * p["K"]) / (p["alpha"] + p["xi"])
    return float(S_star), float(R_star)


S_star, R_star = homogeneous_equilibrium(p)
I_star = 0.0
P_star = 1.0
A_star = 0.0


# ----------------------------
# 6) Reactions (same as base)
# ----------------------------
def compute_reactions(S: np.ndarray, R: np.ndarray, I: np.ndarray, P: np.ndarray, A: np.ndarray, p: dict):
    phi_I = phi(I)
    delta_I = delta(I, p)
    sat = 1.0 - (S + R) / p["K"]

    G_S = (
        p["lambda_S"] * S * sat
        - p["alpha"] * S
        - delta_I * S
        + p["xi"] * (1.0 - phi_I) * R
    )

    G_R = (
        p["lambda_R"] * R * sat
        + p["alpha"] * S
        + p["eta"] * phi_I * A * R
        - p["xi"] * (1.0 - phi_I) * R
    )

    G_I = -p["gamma_I"] * I

    G_P = -p["theta"] * phi_I * P + p["beta"] * (1.0 - phi_I) * A
    G_A =  p["theta"] * phi_I * P - p["beta"] * (1.0 - phi_I) * A

    return G_S, G_R, G_I, G_P, G_A


# ----------------------------
# 7) Metrics
# ----------------------------
def l2_deviation(field: np.ndarray, const_value: float, h: float) -> float:
    diff = field - const_value
    return float(np.sqrt(np.sum(diff * diff) * h * h))


# ----------------------------
# 8) Runner: base model (chi=0) for overlay
# ----------------------------
def run_base(T_final: float):
    p_base = dict(p)
    p_base["chi_S"] = 0.0
    p_base["chi_R"] = 0.0
    # in base regime, keep A constant at A*=0 (as in your Section 5 text)
    p_base["d_A"] = 0.0
    p_base["gamma_A"] = 0.0

    solve_S, rhs_S = cn_factorized(L_h, p_base["d_S"], dt)
    solve_R, rhs_R = cn_factorized(L_h, p_base["d_R"], dt)
    solve_I, rhs_I = cn_factorized(L_h, p_base["d_I"], dt)

    rng = np.random.default_rng(seed)

    S = S_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    R = R_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    I = I_star + eps * rng.uniform(0.0, 1.0, size=(Ny, Nx))
    P = P_star * np.ones((Ny, Nx))
    A = A_star * np.ones((Ny, Nx))

    nsteps = int(round(T_final / dt))

    times = []
    E_S = []

    for n in range(nsteps + 1):
        t = n * dt
        times.append(t)
        E_S.append(l2_deviation(S, S_star, h))
        if n == nsteps:
            break

        G_S, G_R, G_I, G_P, G_A = compute_reactions(S, R, I, P, A, p_base)

        # CN diffusion
        S_flat = S.reshape(-1)
        R_flat = R.reshape(-1)
        I_flat = I.reshape(-1)

        rhs_vec_S = rhs_S.dot(S_flat) + dt * G_S.reshape(-1)
        rhs_vec_R = rhs_R.dot(R_flat) + dt * G_R.reshape(-1)
        rhs_vec_I = rhs_I.dot(I_flat) + dt * G_I.reshape(-1)

        S = solve_S(rhs_vec_S).reshape((Ny, Nx))
        R = solve_R(rhs_vec_R).reshape((Ny, Nx))
        I = solve_I(rhs_vec_I).reshape((Ny, Nx))

        # ODE for P,A (but A stays at zero here because G_A uses A and P only)
        P = np.maximum(P + dt * G_P, 0.0)
        A = np.maximum(A + dt * G_A, 0.0)

        S = np.maximum(S, 0.0)
        R = np.maximum(R, 0.0)
        I = np.maximum(I, 0.0)

    return np.array(times), np.array(E_S)


# ----------------------------
# 9) Runner: unidirectional chemotaxis (open-loop A relaxes)
# ----------------------------
def run_oneway(T_final: float):
    solve_S, rhs_S = cn_factorized(L_h, p["d_S"], dt)
    solve_R, rhs_R = cn_factorized(L_h, p["d_R"], dt)
    solve_I, rhs_I = cn_factorized(L_h, p["d_I"], dt)

    # Signal A: CN diffusion + explicit decay (open-loop)
    solve_A, rhs_A = cn_factorized(L_h, p["d_A"], dt)

    rng = np.random.default_rng(seed)

    S = S_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    R = R_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    I = I_star + eps * rng.uniform(0.0, 1.0, size=(Ny, Nx))
    P = P_star * np.ones((Ny, Nx))

    # Initialize A with unbiased noise so taxis has something to climb
    # (this matches the narrative: "relaxation of imposed heterogeneity")
    A = A_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    # enforce nonnegativity if you interpret A as a concentration
    A = np.maximum(A, 0.0)

    nsteps = int(round(T_final / dt))

    times = []
    E_S = []

    for n in range(nsteps + 1):
        t = n * dt
        times.append(t)
        E_S.append(l2_deviation(S, S_star, h))
        if n == nsteps:
            break

        # base reactions for (S,R,I,P) use current A but do not create A-feedback
        G_S, G_R, G_I, G_P, _G_A_unused = compute_reactions(S, R, I, P, A, p)

        # unidirectional taxis terms (linear)
        C_S = chemotaxis_linear(S, A, p["chi_S"], h)
        C_R = chemotaxis_linear(R, A, p["chi_R"], h)

        # CN diffusion for S,R,I
        S_flat = S.reshape(-1)
        R_flat = R.reshape(-1)
        I_flat = I.reshape(-1)

        rhs_vec_S = rhs_S.dot(S_flat) + dt * (G_S + C_S).reshape(-1)
        rhs_vec_R = rhs_R.dot(R_flat) + dt * (G_R + C_R).reshape(-1)
        rhs_vec_I = rhs_I.dot(I_flat) + dt * G_I.reshape(-1)

        S = solve_S(rhs_vec_S).reshape((Ny, Nx))
        R = solve_R(rhs_vec_R).reshape((Ny, Nx))
        I = solve_I(rhs_vec_I).reshape((Ny, Nx))

        # Open-loop signal relaxation: A_t = d_A ΔA - gamma_A A
        A_flat = A.reshape(-1)
        rhs_vec_A = rhs_A.dot(A_flat) + dt * (-p["gamma_A"] * A_flat)
        A = solve_A(rhs_vec_A).reshape((Ny, Nx))

        # ODE for P (A evolves independently here)
        P = P + dt * G_P

        # positivity
        S = np.maximum(S, 0.0)
        R = np.maximum(R, 0.0)
        I = np.maximum(I, 0.0)
        P = np.maximum(P, 0.0)
        A = np.maximum(A, 0.0)

    return np.array(times), np.array(E_S)


# ----------------------------
# 10) Run and plot overlay (blue solid vs red dashed)
# ----------------------------
t_base, E_base = run_base(T_final)
t_oneway, E_oneway = run_oneway(T_final)

plt.figure(figsize=(6.4, 4.4))
plt.semilogy(t_base, E_base, linewidth=2, label="Base model (Regime I)")
plt.semilogy(t_oneway, E_oneway, "r--", linewidth=2, label="Unidirectional (Regime I$'$)")
plt.xlabel("t")
plt.ylabel(r"$E_S(t)=\|S(\cdot,t)-S^*\|_{L^2(U)}$")
plt.title("Deviation norm: base vs unidirectional chemotaxis")
plt.grid(True, which="both", alpha=0.3)
plt.legend()
savefig(FIG_NORM_OVERLAY)

print(f"Saved: {FIG_NORM_OVERLAY}")

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2977439619.py:5: SyntaxWarning: invalid escape sequence '\s'
  \subsection{Unidirectional chemotaxis: transient focusing without generation (Regime I')}


Saved: figures/fig_norm_evolution_oneway_overlay.png


In [9]:
"""
code_regimeII_III_bifeedback.py

Reproducible code for:
  \subsection{Bidirectional feedback: finite-band patterns vs aggregation (Regimes II--III)}
  \label{subsec:numerical_bifeedback}

Model (minimal bidirectional feedback; keep (R,I,P,A) as in one-way setting):
  - S follows gradients of c (and optionally A, but we set chi_S=chi_R=0 here to
    isolate the S-c loop as in your revised Section 5 plan).
  - c is produced by S and decays:
        c_t = D_c Δc + kappa S - rho' c.

Regime II (well-posed patterns):
  - Use flux saturation / velocity limiting consistent with Eq. (flux_saturation_c):
        -∇·( chi_S' * S * ∇c / (1 + alpha_c |∇c|) ).

Regime III (aggregation / breakdown):
  - Use the linear taxis flux:
        -∇·( chi_S' * S * ∇c ).

Implementation:
  - Neumann BC via ghost-point reflection for diffusion (Kronecker Laplacian).
  - Neumann-consistent gradient/divergence for taxis using reflect padding.
  - Semi-implicit CN for diffusion; explicit for reaction and taxis.
    For c we use CN diffusion + explicit reaction/source; this is adequate for dt
    used here (you can make c fully implicit if desired, but not necessary).

Outputs:
  figures/fig_regimeII_patterns_S_c.png     (analog of Fig \label{fig:chemotaxis_attractant})
  figures/fig_regimeIII_breakdown_trace.png (diagnostic; detect overflow/NaN)
  figures/fig_regimeIII_snapshot_S.png      (if it survives long enough)

Run:
  python code_regimeII_III_bifeedback.py

Notes for paper consistency:
  - Symbols are I-only (no D, gamma_d, K_D).
  - This script assumes you already have phi(I), delta(I) and base reactions
    from your codebase. They are included here for a self-contained run.
  - If your manuscript keeps (R,I,P,A) in the background for this experiment,
    you may set their dynamics to baseline; we keep them but set chi_R=0 and
    keep A fixed at 0 to isolate the feedback loop, matching your Section 5 text.
"""

from __future__ import annotations
import os
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt


# ----------------------------
# 0) I/O
# ----------------------------
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

FIG_REGIMEII = os.path.join(FIGDIR, "fig_regimeII_patterns_S_c.png")
FIG_REGIMEIII_TRACE = os.path.join(FIGDIR, "fig_regimeIII_breakdown_trace.png")
FIG_REGIMEIII_SNAP = os.path.join(FIGDIR, "fig_regimeIII_snapshot_S.png")


def savefig(path: str) -> None:
    plt.tight_layout()
    plt.savefig(path, dpi=300)
    plt.close()


# ----------------------------
# 1) Parameters (baseline + feedback)
# ----------------------------
p = {
    # Diffusion coefficients (baseline)
    "d_S": 1e-3,
    "d_R": 1e-3,
    "d_I": 5.0,

    # Baseline: d_P=d_A=0 in main model; we keep A fixed anyway.
    # We allow a small diffusion for conversion variables if needed,
    # but do NOT use them for this bifeedback loop.
    "d_P": 0.0,
    "d_A": 0.0,

    # Kinetics
    "lambda_S": 0.5,
    "lambda_R": 0.5,
    "K": 2.0,
    "alpha": 0.1,
    "xi": 0.1,

    # Drug (single-dose open-loop)
    "gamma_I": 1.0,
    "delta_0": 0.5,
    "K_I": 0.5,

    # Microenvironment conversion / resistance modulation
    "eta": 0.2,
    "theta": 1.0,
    "beta": 0.5,

    # Chemotaxis baseline (set to 0 to isolate S-c loop)
    "chi_S": 0.0,
    "chi_R": 0.0,

    # Bidirectional feedback (S -> c -> drift of S)
    "chi_S_prime": 0.1,   # chi'_S
    "D_c": 1e-2,          # diffusion of c (D_c)
    "kappa": 0.1,         # production rate
    "rho_prime": 0.05,    # decay rate

    # Flux saturation strength alpha_c for Regime II (velocity limiting)
    "alpha_c": 0.25,      # default; see your discussion for recommended range
}


# ----------------------------
# 2) Domain / grid / time
# ----------------------------
L = 1.0
Nx = Ny = 101
h = L / (Nx - 1)

dt = 1e-2
T_final = 50.0

seed = 101
eps = 5e-2


# ----------------------------
# 3) Neumann Laplacian (ghost reflection)
# ----------------------------
def neumann_laplacian_1d(N: int, h: float) -> sp.csr_matrix:
    e = np.ones(N)
    A = sp.diags([e, -2 * e, e], [-1, 0, 1], shape=(N, N), format="lil")
    A[0, 1] = 2.0
    A[N - 1, N - 2] = 2.0
    return (A.tocsr()) / (h * h)


def neumann_laplacian_2d(Nx: int, Ny: int, h: float) -> sp.csr_matrix:
    Ax = neumann_laplacian_1d(Nx, h)
    Ay = neumann_laplacian_1d(Ny, h)
    Ix = sp.eye(Nx, format="csr")
    Iy = sp.eye(Ny, format="csr")
    return sp.kron(Iy, Ax, format="csr") + sp.kron(Ay, Ix, format="csr")


L_h = neumann_laplacian_2d(Nx, Ny, h)
N = Nx * Ny
Isp = sp.eye(N, format="csr")


def cn_factorized(Lh: sp.csr_matrix, d: float, dt: float) -> tuple[callable, sp.csr_matrix]:
    lhs = (Isp - 0.5 * dt * d * Lh).tocsc()
    rhs = (Isp + 0.5 * dt * d * Lh).tocsr()
    solve = spla.factorized(lhs)
    return solve, rhs


# ----------------------------
# 4) Neumann-consistent grad/div for taxis (reflect padding)
# ----------------------------
def grad_neumann(F: np.ndarray, h: float) -> tuple[np.ndarray, np.ndarray]:
    Fp = np.pad(F, pad_width=1, mode="reflect")
    Fx = (Fp[1:-1, 2:] - Fp[1:-1, :-2]) / (2 * h)
    Fy = (Fp[2:, 1:-1] - Fp[:-2, 1:-1]) / (2 * h)
    return Fx, Fy


def div_neumann(Jx: np.ndarray, Jy: np.ndarray, h: float) -> np.ndarray:
    Jxp = np.pad(Jx, pad_width=1, mode="reflect")
    Jyp = np.pad(Jy, pad_width=1, mode="reflect")
    dJx = (Jxp[1:-1, 2:] - Jxp[1:-1, :-2]) / (2 * h)
    dJy = (Jyp[2:, 1:-1] - Jyp[:-2, 1:-1]) / (2 * h)
    return dJx + dJy


def taxis_linear_S_to_c(S: np.ndarray, c: np.ndarray, chi_prime: float, h: float) -> np.ndarray:
    """
    Linear taxis:
      -∇·(chi' * S * ∇c).
    """
    cx, cy = grad_neumann(c, h)
    Jx = chi_prime * S * cx
    Jy = chi_prime * S * cy
    return -div_neumann(Jx, Jy, h)


def taxis_flux_saturated_S_to_c(S: np.ndarray, c: np.ndarray, chi_prime: float, alpha_c: float, h: float) -> np.ndarray:
    """
    Flux saturation / velocity limiting:
      -∇·( chi' * S * ∇c / (1 + alpha_c * |∇c|) ).
    """
    cx, cy = grad_neumann(c, h)
    grad_norm = np.sqrt(cx * cx + cy * cy)
    denom = 1.0 + alpha_c * grad_norm
    Jx = chi_prime * S * (cx / denom)
    Jy = chi_prime * S * (cy / denom)
    return -div_neumann(Jx, Jy, h)


# ----------------------------
# 5) Nonlinearities and base reactions (I-only)
# ----------------------------
def phi(I: np.ndarray) -> np.ndarray:
    return np.tanh(5.0 * I)


def delta(I: np.ndarray, p: dict) -> np.ndarray:
    return p["delta_0"] * I / (I + p["K_I"] + 1e-12)


def homogeneous_equilibrium(p: dict) -> tuple[float, float]:
    S_star = (p["xi"] * p["K"]) / (p["alpha"] + p["xi"])
    R_star = (p["alpha"] * p["K"]) / (p["alpha"] + p["xi"])
    return float(S_star), float(R_star)


S_star, R_star = homogeneous_equilibrium(p)


def compute_reactions_base(S: np.ndarray, R: np.ndarray, I: np.ndarray, P: np.ndarray, A: np.ndarray, p: dict):
    """
    Same reaction terms as base model (used here to keep (R,I,P,A) consistent).
    In this bifeedback experiment we keep A fixed at 0, and we focus on S-c feedback.
    """
    phi_I = phi(I)
    delta_I = delta(I, p)
    sat = 1.0 - (S + R) / p["K"]

    G_S = (
        p["lambda_S"] * S * sat
        - p["alpha"] * S
        - delta_I * S
        + p["xi"] * (1.0 - phi_I) * R
    )
    G_R = (
        p["lambda_R"] * R * sat
        + p["alpha"] * S
        + p["eta"] * phi_I * A * R
        - p["xi"] * (1.0 - phi_I) * R
    )
    G_I = -p["gamma_I"] * I
    G_P = -p["theta"] * phi_I * P + p["beta"] * (1.0 - phi_I) * A
    G_A =  p["theta"] * phi_I * P - p["beta"] * (1.0 - phi_I) * A
    return G_S, G_R, G_I, G_P, G_A


# ----------------------------
# 6) Bidirectional feedback dynamics for c
# ----------------------------
def compute_reaction_c(S: np.ndarray, c: np.ndarray, p: dict) -> np.ndarray:
    return p["kappa"] * S - p["rho_prime"] * c


# ----------------------------
# 7) Helpers for diagnostics / plotting
# ----------------------------
def cross_section_mid(field: np.ndarray) -> np.ndarray:
    return field[Ny // 2, :].copy()


def normalize_to(field: np.ndarray, target: np.ndarray) -> np.ndarray:
    """
    Affine-rescale field to target's min/max (for overlay cross-section plots).
    """
    fmin, fmax = np.min(field), np.max(field)
    tmin, tmax = np.min(target), np.max(target)
    if fmax - fmin < 1e-12:
        return np.full_like(field, 0.5 * (tmin + tmax))
    return (field - fmin) / (fmax - fmin) * (tmax - tmin) + tmin


# ----------------------------
# 8) Core simulator: choose taxis operator (Regime II vs III)
# ----------------------------
def run_bifeedback(T_final: float, saturated: bool) -> dict:
    """
    Run the S-c feedback system.
    If saturated=True: Regime II (flux saturation).
    If saturated=False: Regime III attempt (linear taxis; may blow up).

    Returns a dict with fields and diagnostics.
    """
    rng = np.random.default_rng(seed)

    # Initialize near homogeneous equilibrium + unbiased noise
    S = S_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    R = R_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    I = np.zeros((Ny, Nx))
    P = np.ones((Ny, Nx))
    A = np.zeros((Ny, Nx))  # fixed at 0 to isolate c-loop

    # Initialize c around its homogeneous equilibrium c* = kappa*S*/rho'
    c_star = p["kappa"] * S_star / p["rho_prime"]
    c = c_star + eps * rng.uniform(-1.0, 1.0, size=(Ny, Nx))
    c = np.maximum(c, 0.0)

    # CN solvers
    solve_S, rhs_S = cn_factorized(L_h, p["d_S"], dt)
    solve_R, rhs_R = cn_factorized(L_h, p["d_R"], dt)
    solve_I, rhs_I = cn_factorized(L_h, p["d_I"], dt)
    solve_c, rhs_c = cn_factorized(L_h, p["D_c"], dt)

    nsteps = int(round(T_final / dt))

    maxS_hist = []
    maxc_hist = []
    t_hist = []

    status = "ok"
    blow_step = None

    for n in range(nsteps + 1):
        t = n * dt
        if n % 20 == 0:
            t_hist.append(t)
            maxS_hist.append(float(np.max(S)))
            maxc_hist.append(float(np.max(c)))

        if n == nsteps:
            break

        # Base reactions
        G_S, G_R, G_I, G_P, G_A = compute_reactions_base(S, R, I, P, A, p)

        # Feedback for c
        G_c = compute_reaction_c(S, c, p)

        # Taxis term for S up grad c
        if saturated:
            C_S = taxis_flux_saturated_S_to_c(S, c, p["chi_S_prime"], p["alpha_c"], h)
        else:
            C_S = taxis_linear_S_to_c(S, c, p["chi_S_prime"], h)

        # CN diffusion updates
        S_flat = S.reshape(-1)
        R_flat = R.reshape(-1)
        I_flat = I.reshape(-1)
        c_flat = c.reshape(-1)

        rhs_vec_S = rhs_S.dot(S_flat) + dt * (G_S + C_S).reshape(-1)
        rhs_vec_R = rhs_R.dot(R_flat) + dt * (G_R).reshape(-1)
        rhs_vec_I = rhs_I.dot(I_flat) + dt * (G_I).reshape(-1)

        rhs_vec_c = rhs_c.dot(c_flat) + dt * (G_c).reshape(-1)

        S = solve_S(rhs_vec_S).reshape((Ny, Nx))
        R = solve_R(rhs_vec_R).reshape((Ny, Nx))
        I = solve_I(rhs_vec_I).reshape((Ny, Nx))
        c = solve_c(rhs_vec_c).reshape((Ny, Nx))

        # ODE updates (no diffusion here)
        P = P + dt * G_P
        # A stays at 0 (or you may evolve by its ODE if you prefer)
        # A = A + dt * G_A

        # positivity
        S = np.maximum(S, 0.0)
        R = np.maximum(R, 0.0)
        I = np.maximum(I, 0.0)
        P = np.maximum(P, 0.0)
        c = np.maximum(c, 0.0)

        # breakdown detection (Regime III)
        if (not np.isfinite(S).all()) or (not np.isfinite(c).all()) or np.max(S) > 1e12 or np.max(c) > 1e12:
            status = "breakdown"
            blow_step = n
            break

    return {
        "S": S,
        "c": c,
        "status": status,
        "blow_step": blow_step,
        "t_hist": np.array(t_hist),
        "maxS_hist": np.array(maxS_hist),
        "maxc_hist": np.array(maxc_hist),
    }


# ----------------------------
# 9) Regime II run (flux saturation): plot S, c, cross-section
# ----------------------------
out2 = run_bifeedback(T_final=T_final, saturated=True)

S2 = out2["S"]
c2 = out2["c"]

plt.figure(figsize=(12.5, 4.0))

plt.subplot(1, 3, 1)
im1 = plt.imshow(S2, origin="lower", extent=[0, L, 0, L])
plt.colorbar(im1, fraction=0.046, pad=0.04)
plt.title("S (Regime II: flux-saturated)")

plt.subplot(1, 3, 2)
im2 = plt.imshow(c2, origin="lower", extent=[0, L, 0, L])
plt.colorbar(im2, fraction=0.046, pad=0.04)
plt.title("c (produced by S)")

plt.subplot(1, 3, 3)
xs = np.linspace(0, L, Nx)
S_slice = cross_section_mid(S2)
c_slice = cross_section_mid(c2)
plt.plot(xs, S_slice, linewidth=2, label="S")
plt.plot(xs, normalize_to(c_slice, S_slice), "--", linewidth=2, label="c (scaled)")
plt.title("Cross-section at y=0.5")
plt.grid(True, alpha=0.3)
plt.legend()

savefig(FIG_REGIMEII)
print(f"Saved: {FIG_REGIMEII}")


# ----------------------------
# 10) Regime III attempt (linear taxis): plot breakdown trace
# ----------------------------
out3 = run_bifeedback(T_final=T_final, saturated=False)

plt.figure(figsize=(6.6, 4.4))
plt.plot(out3["t_hist"], out3["maxS_hist"], linewidth=2, label="max S")
plt.plot(out3["t_hist"], out3["maxc_hist"], linewidth=2, label="max c")
plt.yscale("log")
plt.xlabel("t")
plt.ylabel("max value (log scale)")
title = "Regime III (linear taxis): "
if out3["status"] == "breakdown":
    title += f"breakdown at step {out3['blow_step']} (t≈{out3['blow_step']*dt:.2f})"
else:
    title += "no breakdown within T_final (increase chi'_S to trigger)"
plt.title(title)
plt.grid(True, which="both", alpha=0.3)
plt.legend()
savefig(FIG_REGIMEIII_TRACE)
print(f"Saved: {FIG_REGIMEIII_TRACE}")

# Optional snapshot if it didn't explode immediately
if out3["status"] != "breakdown":
    plt.figure(figsize=(5.4, 4.6))
    im = plt.imshow(out3["S"], origin="lower", extent=[0, L, 0, L])
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.title("S snapshot (linear taxis; no breakdown yet)")
    savefig(FIG_REGIMEIII_SNAP)
    print(f"Saved: {FIG_REGIMEIII_SNAP}")

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4264222699.py:5: SyntaxWarning: invalid escape sequence '\s'
  \subsection{Bidirectional feedback: finite-band patterns vs aggregation (Regimes II--III)}


Saved: figures/fig_regimeII_patterns_S_c.png
Saved: figures/fig_regimeIII_breakdown_trace.png
Saved: figures/fig_regimeIII_snapshot_S.png
